In [17]:
import pandas as pd
import numpy as np

In [18]:
# load csv containing zipcodes and clean the file before merging
zipcodes = pd.read_csv('zipcodes.csv')
zipcodes['City'] = zipcodes.apply(lambda row: row['City'] if row['County'] != 'Queens' else row['County'], axis = 1) 
zipcodes = zipcodes.ix[:, :2]
zipcode_key = zipcodes.set_index('Zip Code').T.to_dict('list')

In [19]:
df_buy = pd.read_csv('streeteasy_buy_clean_geo.csv')

drops = []
for idx, column in enumerate(df_buy.columns.values):
    #print idx, column
    if 'Unnamed' in column or 'fill' in column:
        drops.append(idx)
df_buy.drop(df_buy.columns[drops], 1, inplace = True)

In [20]:
def get_borough(zipcode):
    try:
        return zipcode_key[int(zipcode)][0]
    except:
        return np.nan

df_buy['Borough'] = df_buy['Zipcode'].apply(lambda zipcode: get_borough(zipcode))
df_buy['Sqft_sq'] = df_buy['Sqft'].apply(lambda sqft: sqft ** 2.0)

In [21]:
df_buy.Borough = df_buy.Borough.replace(to_replace = 'New York', value = 'Manhattan')

In [22]:
df_buy['Borough'].value_counts()

Manhattan        4433
Brooklyn         2099
Queens            616
Bronx             366
Staten Island      77
Name: Borough, dtype: int64

In [23]:
df_buy.columns.values

array(['Address', 'Architect', 'Baths', 'Beds', 'Bike Room',
       'Building Amenities', 'Building Class', 'Built Year',
       'Cats Only - No Dogs', 'Cats and Dogs Allowed',
       "Children's Playroom", 'City', 'City Council', 'Cold Storage',
       'Community', 'Community Recreation Facilities', 'Concierge',
       'Crime', 'Deck', 'Developer', 'Doorman', 'Elevator',
       'Full-time Doorman', 'Garage Parking', 'Garden', 'Green Building',
       'Gym', 'Highlights', 'Hot Tub', 'Interiors', 'Land Lease',
       'Laundry in Building', 'Live-in Super', 'Media Room',
       'NYC Storm Zone 1', 'NYC Storm Zone 2', 'NYC Storm Zone 3',
       'NYC Storm Zone 4', 'NYC Storm Zone 5', 'NYC Storm Zone 6',
       'Package Room', 'Parking Available', 'Part-time Doorman', 'Patio',
       'Pets Allowed', 'Police', 'Price', 'Roof Deck', 'School District',
       'SchoolScore', 'Smoke-free', 'Sqft', 'State', 'Storage Available',
       'Stories', 'Street', 'Swimming Pool', 'Transportation', 'Unit

In [24]:
df_buy_clean = df_buy.drop(['Architect', 'Building Amenities', 'Community', 'City Council', 
            'Developer', 'Highlights', 'Transportation', 'Street', 'State'], axis=1)
df_buy_clean = df_buy_clean[df_buy_clean['Sqft'] < 60000]

In [25]:
df_buy_clean.to_csv('buy_clean_dataset.csv')

In [27]:
# store the predictor dataset for visualization in tableau
df_buy_clean.to_excel('buy_visualization_data/buy_full_dataset.xlsx')